In [1]:
from bs4 import BeautifulSoup
import sys
import os
from pprint import pprint
import argparse
from pytictoc import TicToc
from itertools import chain
import shutil
import re

def makedir(dirpath, mode=0):  # mode=1: show message, mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)

def xml_to_csv_nr_ml1(fin, fout):
    f = open(fin, encoding="utf-8")
    f2 = open(fout, 'w') # _nr_ml1.csv
    # print("nr_ml1 >>>>>")
    # Writing the column names...
    # -------------------------------------------------/
    # columns = ["time", "type_id",
    columns = ["Timestamp", "type_id",
        "Raster ARFCN",
        "Num Cells",
        "Serving Cell Index",
        "Serving Cell PCI",
        ]
    f2.write(",".join(columns)+'\n')

    l = f.readline()

    max_length = 0
    while l:
        if r"<dm_log_packet>" in l:
            soup = BeautifulSoup(l, 'html.parser')
            timestamp = soup.find(key="timestamp").get_text()
            type_id = soup.find(key="type_id").get_text()

            if type_id == '5G_NR_ML1_Searcher_Measurement_Database_Update_Ext':
                arfcn = soup.find(key="Raster ARFCN").get_text()
                num_cells = soup.find(key="Num Cells").get_text()
                serving_cell_idex = soup.find(key="Serving Cell Index").get_text()
                serving_cell_pci = soup.find(key="Serving Cell PCI").get_text()
                pcis = [pci.get_text() for pci in soup.findAll(key="PCI")]
                rsrps = [rsrp.get_text() for rsrp in soup.findAll(key="Cell Quality Rsrp")]
                rsrqs = [rsrq.get_text() for rsrq in soup.findAll(key="Cell Quality Rsrq")]

                A = []
                for i in range(int(num_cells)):    
                    A.append(pcis[i])
                    A.append(rsrps[i])
                    A.append(rsrqs[i])

                x = len([timestamp, type_id, arfcn, num_cells, serving_cell_idex, serving_cell_pci] + A)
                max_length = x if x > max_length else max_length
                f2.write(",".join([timestamp, type_id, arfcn, num_cells, serving_cell_idex, serving_cell_pci] + A)+'\n')
            
            else: # 只處理nr_ml1資料過濾其他type
                while l and r"</dm_log_packet>" not in l:
                    l = f.readline()

            l = f.readline()
            
        else:
            print(l,"Error!")
            delete = True
            break 
            
    f2.close()
    f.close()

    # csv Header process
    with open(fout, 'r') as csvinput:
        new_f = fout[:-4]+"_new.csv"
        l = csvinput.readline()
        x = len(l.split(','))
        X = []
        for i in range(int((max_length-x)/3)):
            X += [f"PCI{i}",f"RSRP{i}",f"RSRQ{i}"]
        X = columns+X
        with open(new_f, 'w') as csvoutput:
            csvoutput.write(",".join(X)+'\n')
            l = csvinput.readline()
            while l:
                csvoutput.write(l)
                l = csvinput.readline()
    os.system(f"rm {fout}") # Remove
    os.system(f"mv {new_f} {fout}") # Rename

In [2]:
path = "./temp"
makedir(path)

mkdir ./temp


In [3]:
database = "/home/wmnlab/D/database/"
# database = "/Users/jackbedford/Desktop/MOXA/Code/data/"
dates = [
    "2023-05-07",
]
exps = {  # experiment_name: (number_of_experiment_rounds, list_of_experiment_round)
            # If the list is None, it will not list as directories.
            # If the list is empty, it will list all directories in the current directory by default.
            # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    "_Bandlock_8_Schemes_Phone": (7, ["#{:02d}".format(i+1) for i in range(7)]),
}
_devices = [
    ["sm00", "sm01", "sm02", "sm03", "sm04", "sm05", "sm06", "sm07",],
]
_schemes = [
    ["All", "B3", "B7", "B8", "B3B7", "B3B8", "B7B8", "LTE",],
]

In [4]:
def fgetter():
    files_collection = []
    tags = "diag_log"
    for filename in filenames:
        if filename.startswith(tags) and filename.endswith(".txt"):
            files_collection.append(filename)
    return files_collection

def main():
    files_collection = fgetter()
    if len(files_collection) == 0:
        print("No candidate file.")
    for filename in files_collection:
        fin = os.path.join(source_dir, filename)
        # fout1 = os.path.join(target_dir, "{}_rrc.csv".format(filename[:-4]))
        # fout2 = os.path.join(target_dir, "{}_ml1.csv".format(filename[:-4]))
        fout3 = os.path.join(target_dir, "{}_nr_ml1.csv".format(filename[:-4]))
        # print(">>>>> convert from '{}' into '{}'...".format(fin, fout1))
        # xml_to_csv_rrc(fin, fout1)
        # savemove(os.path.join(source_dir, "{}_rrc.csv".format(filename[:-4])), target_dir, "{}_rrc.csv".format(filename[:-4]))
        # print(">>>>> convert from '{}' into '{}'...".format(fin, fout2))
        # xml_to_csv_ml1(fin, fout2)
        # savemove(os.path.join(source_dir, "{}_ml1.csv".format(filename[:-4])), target_dir, "{}_ml1.csv".format(filename[:-4]))
        print(">>>>> convert from '{}' into '{}'...".format(fin, fout3))
        xml_to_csv_nr_ml1(fin, fout3)
        # savemove(os.path.join(source_dir, "{}_nr_ml1.csv".format(filename[:-4])), target_dir, "{}_nr_ml1.csv".format(filename[:-4]))
    print()

# ******************************* Check Files *********************************
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        print(os.path.join(database, date, expr))
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("|___ {} does not exist.".format(os.path.join(database, date, expr, dev)))
                continue
            
            print("|___", os.path.join(database, date, expr, dev))
            if traces == None:
                # print(os.path.join(database, date, expr, dev))
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            print("|    ", times)
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            if len(traces) != times:
                print("***************************************************************************************")
                print("Warning: the number of traces does not match the specified number of experiment times.")
                print("***************************************************************************************")
            for trace in traces:
                print("|    |___", os.path.join(database, date, expr, dev, trace))
        print()
# *****************************************************************************

# ******************************** Processing *********************************
t = TicToc()  # create instance of class
t.tic()       # Start timer
err_handles = []
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("{} does not exist.\n".format(os.path.join(database, date, expr, dev)))
                continue

            if traces == None:
                print("------------------------------------------")
                print(date, expr, dev)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev)
                target_dir = os.path.join(database, date, expr, dev)
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            for trace in traces:
                print("------------------------------------------")
                print(date, expr, dev, trace)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev, trace, "middle")
                # target_dir = os.path.join(database, date, expr, dev, trace, "data")
                target_dir = "./temp"
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
t.toc()  # Time elapsed since t.tic()
# *****************************************************************************

/home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone
|___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00
|     7
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#01
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#02
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#03
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#04
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#05
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#06
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#07
|___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm01
|     7
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm01/#01
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Schemes_Phone/sm01/#02
|    |___ /home/wmnlab/D/database/2023-05-07/_Bandlock_8_Sche

: 